## Importing libraries

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['PYTHONHASHSEED'] = '2'
import keras
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, Adam, SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
import matplotlib.pyplot as plt
import tensorflow as tf
tf.random.set_seed(19)
import scipy
import numpy as np
from numpy.random import seed
seed(18)
import random
random.seed(18)

## Dataset and functions preparation

In [2]:
#Inverse Cholesky transformation using 4 outputs predicted by the neural network
def prediction_to_density(flat):
    tau = np.zeros([flat.shape[0], 2, 2], dtype=np.complex_)
    tau[:, 0, 0] = flat[:, 0]
    tau[:, 0, 1] = 0
    tau[:, 1, 0] = flat[:, 1] + 1j*flat[:, 2]
    tau[:, 1, 1] = flat[:, 3]
    rho = np.zeros([tau.shape[0], 2, 2], dtype=np.complex_)
    for i in range(rho.shape[0]):
        rho[i] = np.dot(tau[i], np.conjugate(np.transpose(tau[i])))
        rho[i] = rho[i] / np.trace(rho[i])
    return rho

#Extracting density matrix information to input vector for compound model
def flatten_rho(rho):
    flat_rho = np.zeros([rho.shape[0], 4])
    flat_rho[:, 0] = np.real(rho[:, 0, 0])
    flat_rho[:, 1] = np.real(rho[:, 1, 0])
    flat_rho[:, 2] = np.imag(rho[:, 1, 0])
    flat_rho[:, 3] = np.real(rho[:, 1, 1])
    return flat_rho

#Computing fidelity
def fidelity_array(input_1, input_2):
    fid_int = []
    for i in range(input_1.shape[0]):
        dm_sqrt = scipy.linalg.sqrtm(input_1[i])
        dm_pred = input_2[i]
        in_sqrt = np.linalg.multi_dot([dm_sqrt, dm_pred, dm_sqrt])
        trace = np.trace(scipy.linalg.sqrtm(in_sqrt))
        fidelity = np.real(trace**2)
        fid_int.append(fidelity)
    return np.array(fid_int)

In [3]:
#Loading the dataset of density matrices and control voltages
train_voltage = np.load("Data_files/training_voltages.npy")
val_voltage = np.load("Data_files/validation_voltages.npy")
test_voltage = np.load("Data_files/test_voltages.npy")

train_rho = np.load("Data_files/training_rhos.npy")
val_rho = np.load("Data_files/validation_rhos.npy")
test_rho = np.load("Data_files/test_rhos.npy")

## Direct model evaluation

In [4]:
#Loading the direct model
direct_model = load_model("Direct_model.h5")

In [5]:
#Prediction of the direct model on the test set
test_pred_rho = prediction_to_density(direct_model.predict(test_voltage))

#Evaluation of results
DM_fidelities = fidelity_array(test_rho, test_pred_rho)
print("Average fidelity on the test set:", np.mean(DM_fidelities))
print("")

DM_infidelities = 1-DM_fidelities
print("Average infidelity on the test set:", np.mean(DM_infidelities))
print("The 5th percentile of infidelities:", np.quantile(DM_infidelities, 0.05))
print("The 95th percentile of infidelities:", np.quantile(DM_infidelities, 0.95))
print("")

print("The format of infidelities in the article:")
print("(", np.mean(DM_infidelities), 
      "-", np.mean(DM_infidelities)-np.quantile(DM_infidelities, 0.05), 
      "+", np.quantile(DM_infidelities, 0.95)-np.mean(DM_infidelities), ")")

Average fidelity on the test set: 0.9995721959282508

Average infidelity on the test set: 0.0004278040717492351
The 5th percentile of infidelities: 8.844168712163203e-07
The 95th percentile of infidelities: 0.000279187546582216

The format of infidelities in the article:
( 0.0004278040717492351 - 0.0004269196548780188 + -0.0001486165251670191 )


## Compound model evaluation

In [6]:
#Loading the compound model
compound_model = load_model("Compound_model.h5")

#Prediction of the direct model on the test set
test_pred_rho = prediction_to_density(compound_model.predict(flatten_rho(test_rho)))

In [7]:
#Evaluation of results
CM_fidelities = fidelity_array(test_rho, test_pred_rho)
print("Average fidelity on the test set:", np.mean(CM_fidelities))
print("")

CM_infidelities = 1-CM_fidelities
print("Average infidelity on the test set:", np.mean(CM_infidelities))
print("The 5th percentile of infidelities:", np.quantile(CM_infidelities, 0.05))
print("The 95th percentile of infidelities:", np.quantile(CM_infidelities, 0.95))
print("")

print("The format of infidelities in the article:")
print("(", np.mean(CM_infidelities), 
      "-", np.mean(CM_infidelities)-np.quantile(CM_infidelities, 0.05), 
      "+", np.quantile(CM_infidelities, 0.95)-np.mean(CM_infidelities), ")")

Average fidelity on the test set: 0.9998257245985978

Average infidelity on the test set: 0.00017427540140232793
The 5th percentile of infidelities: 1.1056603449632886e-06
The 95th percentile of infidelities: 0.0006844815922890565

The format of infidelities in the article:
( 0.00017427540140232793 - 0.00017316974105736465 + 0.0005102061908867285 )
